In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Nishi
[nltk_data]     Kapadia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Nishi
[nltk_data]     Kapadia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
#Reading the excel data
df = pd.read_excel(r"C:\College\sem6\NLP\project\text_dataset.xlsx")
df

,Text
0,In The Simpsons Movie released in July 2007 na...
1,Subject: professional logo for you now workin...
2,Subject: get your babies diapers bill paid for...
3,Subject: very uuseful attache how to save on ...
4,"Subject: model effort in houston christian , ..."
...,...
2880,Get the official ENGLAND poly ringtone or colo...
2881,"Subject: perfect logo charset = koi 8 - r "" > ..."
2882,Subject: save your money by getting an oem sof...
2883,Dear where you. Call me


# EDA

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2885 entries, 0 to 2884
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    2885 non-null   object
dtypes: object(1)
memory usage: 22.7+ KB


In [6]:
# check for missing values and remove
print(df.isnull().sum())

Text    0
dtype: int64


In [7]:
# check for duplicate values and remove
print(df.duplicated().sum())
df = df.drop_duplicates(keep='first')
df.shape

95


(2790, 1)

# Preprocessing

In [8]:
# replace unnecessary letters/words
replace_list = ["\\n","\\t","\\r","â","€","™","ð","Ÿ","ðŸ‘","$","â€™ll","ƒ","¢","â€ƒ","â€¢","Â§","§","Â","Ã¼","Ã","¼","º","œ","˜","£","â€“","â€œ","&lt;#&gt;","â€Œ","ðŸŽ","ð","Ÿ","Ž","Í","â€Œ ï»¿ Í","ï»¿","ðŸ”¨","ðŸ¤©","©","¤","±","ðŸ˜±","ðŸ˜±"," Í â€Œ ï»¿","ÿ","x = = x","*"]
for letter in replace_list:
    df['Text'] = df['Text'].str.replace(letter, '')

C:\Users\Nishi Kapadia\AppData\Local\Temp\ipykernel_20240\3520636091.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].str.replace(letter, '')


In [9]:
email_replace_list = ["Subject:","cc:"]
for letter in email_replace_list:
    df['Text'] = df['Text'].str.replace(letter, '')

C:\Users\Nishi Kapadia\AppData\Local\Temp\ipykernel_20240\1837515156.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].str.replace(letter, '')


In [15]:
lemmatizer = WordNetLemmatizer()

def preprocessing_text(text):
    if isinstance(text, str):
        text = nltk.word_tokenize(text)

        filtered_words = []
        for word in text:
            if word.isalnum():
                filtered_words.append(word)

        text = filtered_words[:]
        
        lemmatized_words= []
        for word, pos in nltk.pos_tag(text):
            if word.isalnum() and word not in stopwords.words('english') and word not in string.punctuation:
                # Get the first letter of the POS tag
                pos = pos[0].lower()
                # Consider only adjectives, nouns, and verbs for lemmatization
                if pos in {'a', 'n', 'v'}:
                    lemmatized_words.append(lemmatizer.lemmatize(word, pos))
                else:
                    lemmatized_words.append(word)

        return " ".join(lemmatized_words)
    else:
        # Convert non-string types to string
        return str(text)

# Apply the modified transform_text2 function using lambda to the 'Text' column
df['preprocessed_text'] = df['Text'].apply(lambda x: preprocessing_text(x))
df['preprocessed_text']

C:\Users\Nishi Kapadia\AppData\Local\Temp\ipykernel_20240\2202711307.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preprocessed_text'] = df['Text'].apply(lambda x: preprocessing_text(x))


0       In The Simpsons Movie release July 2007 name b...
1       professional logo work company image start vis...
2       get baby diaper bill pay year family could def...
3       uuseful attache save medlcations 70 pharmsho f...
4       model effort houston christian spring fall win...
                              ...                        
2879    Get official ENGLAND poly ringtone colour flag...
2881    perfect logo charset koi 8 r thinking breathe ...
2882    save money get oem software need software pc v...
2883                                            Dear Call
2884    logo stationer website design much lt really h...
Name: preprocessed_text, Length: 2790, dtype: object